In [1]:
import numpy as np
import pyvista as pv

In [2]:
num_file = 2

holder_pinn = pv.read("solution_holder_contacts.%d.vtk"%(num_file))
holder_ccx = pv.read("holder.%d.vtk"%(num_file))
holder_ccx2pinn = holder_pinn.sample(holder_ccx, tolerance=0.001, pass_cell_data=True, pass_point_data=True, categorical=False, progress_bar=False, locator=None, pass_field_data=True, mark_blank=True, snap_to_closest_point=True)
holder_ccx2pinn.point_data["U_diff"] = holder_ccx2pinn.point_data["displacement"] - holder_ccx2pinn.point_data["U"]
holder_press = holder_ccx.clip_box([-10.0,10.0,-1.0,1,-10.0,10.0], invert=True)
holder_press.save("holder_rigids.%d.vtk"%(num_file))
holder_ccx2pinn.save("holder_solution_combined.%d.vtk"%(num_file))

In [3]:
num_n = np.array([len(holder_ccx2pinn.point_neighbors(i)) for i in range(holder_ccx2pinn.points.shape[0])])
holder_ccx2pinn.point_data["U_diff"][num_n != 26,:] = 0.0 
holder_ccx2pinn.point_data["connections"] = num_n.astype(np.float64)
holder_ccx2pinn.save("holder_solution_combined.%d.vtk"%(num_file))


In [4]:
print(np.unique(num_n))

[ 7 11 15 17 23 26]


In [ ]:
err_l2_rel = np.linalg.norm(holder_ccx2pinn.point_data["U_diff"]) / np.linalg.norm(holder_ccx2pinn.point_data["U"])
print("l2 relative error", err_l2_rel)
err_linf_rel = np.linalg.norm(holder_ccx2pinn.point_data["U_diff"], ord=np.inf) / np.linalg.norm(holder_ccx2pinn.point_data["U"], ord=np.inf)
print("linf relative error", err_linf_rel)
err_msqe_rel = np.sqrt(np.mean(holder_ccx2pinn.point_data["U_diff"]**2)) / np.linalg.norm(holder_ccx2pinn.point_data["U"], ord=np.inf)
print("mse relative", err_msqe_rel)

l2 relative error 0.04533260453507817
linf relative error 0.059283881598567185
mse relative 0.009857300174274745


: 

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(holder_ccx2pinn)
plotter.show()